In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

In [ ]:
df=pd.read_csv('/content/gdrive/MyDrive/learningdeep/DL_Tutorial-master/DL_Tutorial-master/data/SPECTF.dat')
df.info()

In [ ]:
def evaluatePerformance():
    '''
    Đdánh giá hiệu suất của cây quyết định
    trung bình qua 100 lần thử nghiệm 10-fold cross-validation trên tập dữ liệu SPECTF
    
    Return:
      Ma trận chứa các thông số hiệu suất như sau:
      stats[0,0] = độ chính xác trung bình của cây quyết định
      stats[0,1] = độ lệch chuẩn của độ chính xác của cây quyết định
      stats[1,0] = độ chính xác trung bình của cây quyết định đơn
      stats[1,1] = độ lệch chuẩn của độ chính xác của cây quyết định đơn
      stats[2,0] = độ chính xác trung bình của cây quyết định có 3 cấp
      stats[2,1] = độ lệch chuẩn của độ chính xác của cây quyết định có 3 cấp
      
    ** Lưu ý rằng bạn phải tuân thủ API như đã định nghĩa**
    '''
    
    # Load dữ liệu
    #filename = 'data/SPECTF.dat'
    #data = np.loadtxt(uploaded, delimiter=',')
    #data = [line.split(',') for line in data]
    #data = np.genfromtxt(uploaded, delimiter=',')
    #X = data[:, 1:]
    #y = data[:, 0]
    X=np.asarray(df.drop("1",axis=1))
    y=np.asarray(df["1"])
    n,d = X.shape

    # Số lần thử nghiệm và số lượt cross-validation
    num_trials = 100
    num_folds = 10

    # Mảng để lưu kết quả độ chính xác từ các lượt cross-validation
    decision_tree_acc = np.zeros(num_trials * num_folds)
    decision_stump_acc = np.zeros(num_trials * num_folds)
    dt3_acc = np.zeros(num_trials * num_folds)

    # Thực hiện thử nghiệm
    for trial in range(num_trials):
        # Shuffle dữ liệu
        idx = np.random.permutation(n)
        X = X[idx]
        y = y[idx]

        fold_size = n // num_folds

        # Thực hiện lượt cross-validation
        for fold in range(num_folds):
            # Chia dữ liệu thành tập train và tập test
            test_start = fold * fold_size
            test_end = (fold + 1) * fold_size
            X_test = X[test_start:test_end]
            y_test = y[test_start:test_end]
            X_train = np.concatenate((X[:test_start], X[test_end:]))
            y_train = np.concatenate((y[:test_start], y[test_end:]))

            # Huấn luyện cây quyết định
            clf = DecisionTreeClassifier()
            clf.fit(X_train, y_train)

            # Đánh giá độ chính xác trên tập test
            y_pred = clf.predict(X_test)
            decision_tree_acc[trial * num_folds + fold] = accuracy_score(y_test, y_pred)

            # Huấn luyện cây quyết định đơn
            clf_stump = DecisionTreeClassifier(max_depth=1)
            clf_stump.fit(X_train, y_train)
            y_pred_stump = clf_stump.predict(X_test)
            decision_stump_acc[trial * num_folds + fold] = accuracy_score(y_test, y_pred_stump)

            # Huấn luyện cây quyết định có 3 cấp
            clf_dt3 = DecisionTreeClassifier(max_depth=3)
            clf_dt3.fit(X_train, y_train)
            y_pred_dt3 = clf_dt3.predict(X_test)
            dt3_acc[trial * num_folds + fold] = accuracy_score(y_test, y_pred_dt3)

    # Tính độ chính xác trung bình và độ lệch chuẩn
    mean_decision_tree_acc = np.mean(decision_tree_acc)
    std_decision_tree_acc = np.std(decision_tree_acc)

    mean_decision_stump_acc = np.mean(decision_stump_acc)
    std_decision_stump_acc = np.std(decision_stump_acc)

    mean_dt3_acc = np.mean(dt3_acc)
    std_dt3_acc = np.std(dt3_acc)

    # Tạo ma trận chứa kết quả
    stats = np.zeros((3, 2))
    stats[0, 0] = mean_decision_tree_acc
    stats[0, 1] = std_decision_tree_acc
    stats[1, 0] = mean_decision_stump_acc
    stats[1, 1] = std_decision_stump_acc
    stats[2, 0] = mean_dt3_acc
    stats[2, 1] = std_dt3_acc

    return stats